# Line plots and interactivity 
# with Matplotlib, Seaborn, Plotly, and Altair

In [ ]:
# numerical
import pandas as pd
import numpy as np

# matplotlib
import matplotlib.pyplot as plt

# seaborn
import seaborn as sns
sns.set()
sns.set_context('notebook', font_scale=1.5)
cp = sns.color_palette()

# altair
import altair

# plotly express
import plotly.express as px

## Data

In [ ]:
ts = pd.read_csv('https://raw.githubusercontent.com/benjum/UCLAX-24Fall-EDA/main/Data/ts.csv')
ts.head()

In [ ]:
# casting to datetime is important for
# ensuring plots "just work"

ts.dt = pd.to_datetime(ts.dt)

ts.info()

In [ ]:
ts.head()

In [ ]:
dfp = ts.pivot(index='dt', columns='kind', values='value')
dfp.head()

## Matplotlib

In [ ]:
ts.kind.unique()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(7.5, 5))

for k in ts.kind.unique():
    tmp = ts[ts.kind == k]
    ax.plot(tmp.dt, tmp.value, label=k)

ax.set(xlabel='Date',
       ylabel='Value',
       title='Random Timeseries')    
    
ax.legend(loc=2)
fig.autofmt_xdate()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(7.5, 5))

ax.plot(dfp)

ax.set(xlabel='Date',
       ylabel='Value',
       title='Random Timeseries')

ax.legend(dfp.columns, loc=2)
fig.autofmt_xdate()

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(10, 5))

for i, k in enumerate(ts.kind.unique()):
    ax = plt.subplot(int('22' + str(i + 1)))
    tmp = ts[ts.kind == k]
    ax.plot(tmp.dt, tmp.value, label=k, c=cp[i])

    ax.set(xlabel='Date',
           ylabel='Value',
           title=k)    
    
fig.autofmt_xdate()
fig.tight_layout()

## Seaborn

In [ ]:
ts

In [ ]:
sns.lineplot(data=ts, x='dt', y='value', hue='kind')

In [ ]:
ax = sns.lineplot(data=ts, x='dt', y='value', hue='kind')

ax.set(xlabel='Date',
       ylabel='Value',
       title='Random Timeseries')

ax.get_figure().autofmt_xdate()

In [ ]:
g = sns.FacetGrid(ts, col="kind")
g.map_dataframe(sns.lineplot, x='dt', y='value')

In [ ]:
g = sns.FacetGrid(ts, hue='kind', height=5, aspect=1.5)

# map applies a plotting function to each facet’s subset of the data.
g.map(plt.plot, 'dt', 'value').add_legend()
g.ax.set(xlabel='Date',
         ylabel='Value',
         title='Random Timeseries')
g.fig.autofmt_xdate()

In [ ]:
# g = sns.FacetGrid(ts, row='kind', hue='kind', height=5, aspect=1.5)
g = sns.FacetGrid(ts, col='kind', hue='kind', height=5, aspect=1.5, col_wrap=2)
g.map(plt.plot, 'dt', 'value').add_legend()

g.fig.autofmt_xdate()

## Plotly

https://plotly.com/

In [ ]:
fig = px.line(ts, x="dt", y="value", color='kind')
fig.show()

In [ ]:
fig = px.line(ts.loc[ts['kind'].isin(['A','B'])], x="dt", y="value", color='kind')
fig.show()

In [ ]:
import plotly.express as px

fig = px.line(ts, x="dt", y="value", color='kind')
fig.update_layout(
    width=750,
    height=500,
    plot_bgcolor="White"
)
fig.update_xaxes(showline=True, linecolor='black', showgrid=True, gridwidth=1, gridcolor='LightBlue')
fig.update_yaxes(showline=True, linecolor='black', showgrid=True, gridwidth=1, gridcolor='LightBlue')
fig.show()

In [ ]:
import plotly.express as px

fig = px.line(ts, x="dt", y="value", color='kind')
fig.update_layout(
    width=750,
    height=500,
    plot_bgcolor="White"
)
fig.update_xaxes(showline=True, linecolor='black', showgrid=True, gridwidth=1, gridcolor='LightBlue')
fig.update_yaxes(showline=True, linecolor='black', showgrid=True, gridwidth=1, gridcolor='LightBlue')
fig.write_html('plotly_legendselect.html')

## Altair

https://altair-viz.github.io/

In [ ]:
c = altair.Chart(ts).mark_line().encode(
    x='dt',
    y='value',
    color='kind'
)
c

In [ ]:
c = altair.Chart(ts).mark_line().encode(
    x='dt',
    y='value',
    color=altair.Color('kind', scale=altair.Scale(range=cp.as_hex()))
)
c

In [ ]:
selopac = altair.selection_point(fields=['kind'], bind='legend')

c = altair.Chart(ts).mark_line().encode(
    x='dt',
    y='value',
    color=altair.Color('kind', scale=altair.Scale(range=cp.as_hex())),
    opacity=altair.condition(selopac, altair.value(1), altair.value(0.2))
).add_params(
    selopac
)
c

In [ ]:
selopac = altair.selection_point(fields=['kind'], bind='legend')

c = altair.Chart(ts).mark_line().encode(
    x='dt',
    y='value',
    color=altair.Color('kind', scale=altair.Scale(range=cp.as_hex())),
    opacity=altair.condition(selopac, altair.value(1), altair.value(0.2))
).add_params(
    selopac
)
c.save('altair_legendselect.html')

In [ ]:
# Create a selection that chooses the nearest point & selects based on x-value
nearest = altair.selection_point(nearest=True, on='mouseover', fields=['dt'])


c = altair.Chart(ts).mark_line().encode(
    x='dt',
    y='value',
    color=altair.Color('kind', scale=altair.Scale(range=cp.as_hex()))
)

# Transparent selectors across the chart. This is what tells us
# the x-value of the cursor
selectors = altair.Chart(ts).mark_point().encode(
    x='dt',
    opacity=altair.value(0),
).add_params(
    nearest
)

# Draw points on the line, and highlight based on selection
points = c.mark_point().encode(
    opacity=altair.condition(nearest, altair.value(1), altair.value(0))
)

# Draw text labels near the points, and highlight based on selection
text = c.mark_text(align='left', dx=5, dy=-5).encode(
    text=altair.condition(nearest, 'value', altair.value(' '))
)

# Draw a rule at the location of the selection
rules = altair.Chart(ts).mark_rule(color='gray').encode(
    x='dt',
).transform_filter(
    nearest
)

# Put the five layers into a chart and bind the data
altair.layer(
    c, selectors, points, rules, text
).properties(
    width=600, height=300
)